In [2]:
import pandas as pd
import numpy as np
import json

# Wczytywanie pliku z przepisami

In [9]:
with open('data/przepisy.json') as json_file:
    recipes = json.load(json_file)

In [10]:
len(recipes)

10175

## Wyciąganie składników z przepisów

In [11]:
ingredients = []

for recipe in recipes:
    for ingredient in recipe['ingredient_list']:
        ingredients.append([ recipe['url'], ingredient['ingredient_name'], 
                            ingredient['ingredient_amount'],
                            ingredient['ingredient_unit']])

In [12]:
len(ingredients)

85290

In [15]:
df = pd.DataFrame(ingredients, columns=["URL", "nazwa", "ilosc", "jednostka"])

In [18]:
df.sample(5)

,URL,nazwa,ilosc,jednostka
19650,https://www.przepisy.pl/przepis/mintaj-zapieka...,tłuszcz do wysmarowania formy,None,None
75142,https://www.przepisy.pl/przepis/pasztet-z-jajek,pieprz,1,szczypta
47232,https://www.przepisy.pl/przepis/salatka-z-kurc...,cebula biała,1,szt
34809,https://www.przepisy.pl/przepis/ryba-na-ostro-...,"filety z białej ryby, np. dorsz",500,g
29177,https://www.przepisy.pl/przepis/zapiekany-loso...,cebula,1,szt


In [48]:
df['nazwa_sm'] = df['nazwa'].map(lambda x: x.lower())

## Co mamy w danych?

In [43]:
df['nazwa'].value_counts().shape

(9373,)

In [49]:
df['nazwa_sm'].value_counts().shape

(9340,)

In [37]:
df['ilosc'].value_counts().head(10)

1      29735
2      12189
3       4906
4       4315
0,5     3815
100     3399
200     2635
50      1856
500     1600
300     1600
Name: ilosc, dtype: int64

In [40]:
df['jednostka'].value_counts()

szt         27817
g           14780
łyżki        8645
ml           4629
szczypta     4592
łyżka        3788
łyżeczka     2293
szklanki     1823
opak         1747
pęczek       1601
kg           1538
dag          1533
szklanka     1351
łyżeczki     1240
łyżek         864
pęczka        724
szczypty      655
l             626
plastry       295
plastrów      249
garść         105
ziarna         71
łyżeczek       65
puszka         64
cm             51
ziaren         51
słoik          48
pęczki         48
plaster        36
listków        24
szczypt        24
listki         22
szklanek       20
ziarno         18
puszki          9
plastra         8
listek          4
pęczków         2
słoiki          2
Name: jednostka, dtype: int64

## Sprawdzanie podobieństwa nazw produktów

In [28]:
# pip install python-Levenshtein
import difflib
import Levenshtein

In [34]:
difflib.SequenceMatcher(None, "Pomidory w puszce", "Pomidory krojone").ratio() #the bigger the better

0.6060606060606061

In [33]:
Levenshtein.distance("Pomidory krojone", "pomidory w puszce") #the smaller the better

8

In [41]:
difflib.SequenceMatcher(None, "Pomidory", "Papryka").ratio()

0.4

## Wybranie unikalnych nazw i zapisanie do pomocniczej tabeli

In [51]:
unique_names = df['nazwa_sm'].unique()

In [52]:
unique_names

array(['makaron świderki', 'filet z kurczaka ', 'cebula', ...,
       'pasta z czerwonej papryki', 'ostra i słodka papryka',
       'grzyby mun wcześniej ugotowane'], dtype=object)

In [64]:
def find_similar(ingredient):
    for name in unique_names:
        match = difflib.SequenceMatcher(None, ingredient, name).ratio()
        if match > 0.6:
            print(name, match)

In [76]:
find_similar("szalotka")

szalotki 0.875
mała szalotka 0.7619047619047619
cebule szalotki 0.6086956521739131
szalotka 1.0
cebula szalotka 0.6956521739130435
cebulka szalotka 0.6666666666666666
uszka 0.6153846153846154
salsa 0.6153846153846154
szynka lub łopatka 0.6153846153846154
saletra 0.6666666666666666
kaszanka 0.625
natka 0.6153846153846154
sałatka: 0.625
serwatka 0.625
1 szalotka 0.8888888888888888
szalotka mała  0.7272727272727273


In [77]:
find_similar("pomidor")

pomidor 1.0
pomidory suszone 0.6086956521739131
pomidory 0.9333333333333333
suszone pomidory 0.6086956521739131
sos pomidorowy 0.6666666666666666
pomidory pelati 0.6363636363636364
pomidory pelatti 0.6086956521739131
małe pomidory 0.7
dojrzały pomidor 0.6086956521739131
duże pomidory 0.7
średnie pomidory 0.6086956521739131
sok pomidorowy 0.6666666666666666
pomidory cherry  0.6086956521739131
pomidor suszony 0.6363636363636364
puszka pomidorów 0.6086956521739131
pomidory świeże  0.6086956521739131
pasta pomidorowa 0.6086956521739131
średni pomidor  0.6363636363636364
świeże pomidory 0.6363636363636364
pomidor malinowy 0.6086956521739131
pomidory średnie 0.6086956521739131
pomidory pellati 0.6086956521739131
pomidor średni  0.6363636363636364
pomidory duże  0.6666666666666666
żółty pomidor 0.7
pomidorowe puree 0.6086956521739131
suszony pomidor 0.6363636363636364
duży pomidor  0.7
puree pomidorowe 0.6086956521739131
twardy pomidor 0.6666666666666666
pomidory małe 0.7
świeży pomidor 0.666

## Losowanie składników do analizy

In [78]:
import random

In [85]:
random.choices(unique_names, k=40)

['oliwki zielone, pokrojone w krążki',
 'płatki owsiane błyskawiczne',
 'świeże czerwone chili',
 'duży kawałek pancetty ',
 'malutki ząbek czosnku',
 'orzechówka',
 'ryż do sushi',
 'sos chilli z czosnkiem ',
 'oliwki zielone bez pestek',
 'ostra zielona papryczka',
 'ostra musztarda',
 'czerstwy bochenek chleba tostowego ',
 'pszenny chleb',
 'azjatycka mieszanka warzyw (mrożonych)',
 'jabłka kwaśne ',
 'śmietanka kokosowa ',
 'suszone mięso wołowe',
 'chudy ser twarogowy',
 'kwaśna śmietana 12%',
 '180 g gotowanej szynki (może być parmeńska) pokrojonej w cienkie paseczki',
 'ocet biały winny delikatny np. jabłkowy',
 'groch (najlepiej cały, niełuskany)',
 'płat ciasta francuskiego',
 '2 duże czerwone papryki',
 'oliwki czarne, pokrojone w krążki',
 'pstrąg świeży',
 'filet z morskiej ryby ',
 'bułka tarta lub typu panko',
 'mięso mielone jagnięce',
 'podudzia z kurczaka bez skóry i kości lub piersi z kurczaka',
 'wieprzowina szynkowa w plastrach ',
 'kiełki bambusa z puszki ',
 'pom

## Generowanie listy unikalnych słów które użyte są w nazwach składników

In [99]:
def remove_unwanted_chars(str):
    pass

In [170]:
import re
from operator import itemgetter

words = []
counts = []
stop_words = ["", "z", "w", "lub", "do", "i", "bez", "-", "świeży", "np.", "na", "g", "mała", "np",
             "suszone", "puszki", "pokrojona", "czerwona", "świeże", "zielona", "kilka", "kostkę",
             "pokrojone", "biała", "ze", "żółty", "mały", "świeża", "pęczek", "dekoracja",
             "duże", "mrożone", "knorr"]

# find unique words and count occurences
for name in unique_names:
    for word in re.sub('[^a-ząćęółżźśń ]+', '', name).split(" "):
        if word in stop_words:
            pass
        else:
            if word in words:
                counts[words.index(word)] += 1
            else:
                words.append(word)
                counts.append(1)
                
# create list of words and counts
word_count = []

for i in range(0,len(words)):
    word_count.append([words[i], counts[i]])

# sort the list and reverse it
word_count_sorted = sorted(word_count, key=itemgetter(1))
word_count_sorted.reverse()

len(word_count_sorted)

3516

In [171]:
word_count_sorted

[['ser', 323],
 ['makaron', 215],
 ['mięso', 205],
 ['papryka', 202],
 ['kurczaka', 199],
 ['cebula', 132],
 ['pomidory', 127],
 ['sos', 113],
 ['chili', 112],
 ['filet', 110],
 ['filety', 107],
 ['mielone', 100],
 ['sok', 98],
 ['szynka', 98],
 ['ryż', 97],
 ['grzyby', 95],
 ['puszka', 95],
 ['starty', 93],
 ['plastry', 92],
 ['kości', 91],
 ['śmietana', 90],
 ['małe', 88],
 ['ziemniaki', 88],
 ['fasola', 87],
 ['liście', 86],
 ['posiekana', 85],
 ['kromki', 83],
 ['pieprz', 80],
 ['natka', 79],
 ['papryczka', 78],
 ['pietruszki', 78],
 ['sałata', 77],
 ['chilli', 76],
 ['przyprawa', 75],
 ['najlepiej', 75],
 ['typu', 74],
 ['wędzony', 73],
 ['olej', 72],
 ['kapusta', 71],
 ['kiełbasa', 71],
 ['krewetki', 68],
 ['boczek', 65],
 ['dekoracji', 63],
 ['smaku', 63],
 ['cytryny', 62],
 ['zielone', 61],
 ['mąka', 61],
 ['skóry', 60],
 ['orzechy', 60],
 ['czerwone', 60],
 ['indyka', 60],
 ['chleb', 58],
 ['wieprzowe', 58],
 ['chleba', 57],
 ['ostra', 57],
 ['czosnku', 57],
 ['łososia', 56],
